# to run in all cases
### get the credentials and instantiate the fb_client

### Environmental Variables based on App and User/Page
App M001
 - FB_M001_App_id
 - FB_M001_App_secret
 - FB_M001_user_token
 - FB_M001_DH_App_token (Access toke page)
 - FB_M001_ES_App_token (Access toke page)

ES
 - FB_ES_App_id
 - FB_ES_App_secret
 - FB_ES_user_token
 - FB_ES_ES_App_token (Access toke page)


Pages:
 - FB_DH_Pg_id - DH Page ID: 122097693698010058 
 - FB_ES_Pg_id - ES Page ID: 303403436189223


In [ ]:
import os
from typing import Dict, Optional, Any, List
import json
import facebook
from facebook_api_client import FacebookAPIClient
from post_manager import PostManager
from utils import Utils
from comments_manager import CommentManager

import logging
logging.basicConfig(level=logging.DEBUG)  # Set the logging level to DEBUG

# Load credentials (RELEVANT IF I AM GOING TO USE
# DIFFERENT CREDENTIALS THAT THOSE DEFINED ON THE FacebookAPIClient Class load credentials)

try:
    app_id = os.environ["FB_M001_App_id"]
    app_secret = os.environ["FB_M001_App_secret"]
    access_token = os.environ["FB_M001_DH_App_token"]
    page_id = os.environ["FB_DH_Pg_id"]
    user_id = os.environ["FB_LS_User_id"]

    
except KeyError:
    print("Error: Environment variables not set. Please set them before running the tests.")
    exit(1)  # Exit with error code


# Instantiate the FacebookAPIClient and PostManager
fb_client = FacebookAPIClient(app_id, app_secret, access_token, page_id)
post_manager = PostManager(fb_client)
utils = Utils(fb_client)
comments_manager = CommentManager(fb_client)

In [ ]:
print("app_id", app_id)
print("app_secret", app_secret)
print("access_token", access_token)
print("page_id", page_id)
print("user_id", user_id)

In [ ]:
print(dir(post_manager)) 
print(dir(utils))
print(dir(comments_manager))


## Test to get information about a Group
### problems to get the id of a group - probably scraping better
### problems to get the feed of a Group. Probably scraping better



## Test to grab the latests posts from any of my pages (num_posts)

In [ ]:
try:
    latest_posts = post_manager.get_latest_posts(page_id, num_posts=3)
    
    if latest_posts:  # Check if the list is not empty
        print("Latest posts retrieved successfully:")
        for post in latest_posts:
            print(f"  ID: {post['id']}")
            print(f"  Message: {post.get('message', 'No message available')}")  # Use get() to handle missing messages
            print(f"  Post link: {post['permalink_url']}")
            print(f"  Likes Count: {post['likes']['summary']['total_count']}")
            print(f"  Comments Count: {post['comments']['summary']['total_count']}")
            print("---------------------------------------------------------------------")

            # You can print other fields here as well
            # ----------------------------------------------------------------------------
            # NOTE NOTE NOTE MAY NEED TO Decode message text before printing JSON
            # ----------------------------------------------------------------------------
            
            import json
            print(json.dumps(post, indent=4))      # Pretty-print each post individually

    else:
        print("No latest posts found or error retrieving posts.")
except Exception as e:  # Catching general exceptions for this example
    print(f"Error retrieving latest posts: {e}")



## Test to retrieve a post and its media by post id

In [ ]:

post_id = "303403436189223_122111155826316036"  # Replace with the actual post ID
specific_post = post_manager.get_post_by_id(post_id)
if specific_post:
    import json
    print(json.dumps(specific_post, indent=4)) 

    if "attachments" in specific_post:
        attachments = specific_post["attachments"]["data"]
        for attachment in attachments:
            media_type = attachment.get("media_type")
            if media_type == "photo":
                photo_url = attachment["media"]["image"]["src"]
                print(f"Found photo attachment: {photo_url}")
            elif media_type == "video":
                video_url = attachment["media"]["source"]
                print(f"Found video attachment: {video_url}")
            # Add other media types (e.g., "link", "album") if needed
    else:
        print("No attachments found in this post.")
else:
    print("No post found or error retrieving post.")


# Tests to interact with comments

## Test to read and print the comments of a specific post

In [ ]:
post_id = input("Enter the post ID: ")  # You can prompt for the post ID
    
# Retrieve comments with specified fields
comments = comments_manager.get_post_comments(
    post_id, fields=["id", "message", "created_time", "from", "like_count", "parent", "user_likes"]
)

if comments:
    # Process the retrieved comments
    for comment in comments:
        print(f"Comment ID: {comment['id']}")
        
        # Modified Author Line (handling empty dictionary)
        author_info = comment.get('from', {})  
        if isinstance(author_info, dict) and author_info:  # Check if it's a non-empty dictionary
            author_name = author_info.get('name', "Unknown Author")  # Get name or default
        else:
            author_name = "Unknown Author (ID: Unknown ID)" 

        print(f"Author: {author_name}")

        print(f"Message: {comment['message']}")
        print(f"Created Time: {comment['created_time']}")
        print(f"Likes: {comment.get('like_count', 0)}")

        # Check if it's a reply
        if "parent" in comment:
            print(f"Reply to: {comment['parent']['id']}")
        else:
            print("Not a reply")

        # Check if liked by the user
        if comment.get("user_likes"):
            print("Liked by you")
        else:
            print("Not liked by you")

        print("-" * 20)  # Separator
else:
    print("No comments found or an error occurred.")

In [ ]:
print(comments)


## Test to react to comments

In [ ]:
if comments:
    for comment in comments:
        print(f"Comment ID: {comment['id']}")
        print(f"Message: {comment['message']}")
        
    while True:  # Loop to keep prompting for reactions until user wants to quit
        comment_id_to_react = input("Enter the comment ID to react to (or type 'q' to quit): ")
        if comment_id_to_react.lower() == 'q':
            break

        message = input("Enter the answer text (or press Enter to skip reply): ")
        
        # Improved like input handling
        while True:
            like_input = input("Do you want to like the comment? (y/n): ")
            if like_input.lower() in ['y', 'n']:
                like = like_input.lower() == 'y'
                break
            else:
                print("Invalid input. Please enter 'y' for yes or 'n' for no.")

        response = comments_manager.react_to_comment(comment_id_to_react, message=message if message else None, like=like)
    
        if response:
            print("Reaction successful:", response)
        else:
            print("Failed to react to the comment.")
            

## Test to publish a message post

In [ ]:
   


# Publish a text post
text_post_message = "I loved 2024 Romeria de El Rocio"
try:
    text_post_result = post_manager.publish_text_post(page_id, text_post_message)
except Exception as e:  # Catching general exceptions for this example
    print(f"Error publishing text post: {e}")

print("Text post result:", text_post_result)


## Test to publish a one photo post with a message

In [ ]:


photo_post_message = "El Rocio so full with horses"
photo_path = r"C:\Users\manue\Documents\Disfrutando Huelva\Agrupados para editar\Rocio - Todos\Caballos carros y carretas en el rocio\20240518_131042~2.jpg"
try:
    photo_post_result = post_manager.publish_photo_post(page_id, photo_post_message, photo_path)
except Exception as e:  # Catching general exceptions for this example
    print(f"Error publishing photo post: {e}")

# Print results
    
print("Photo post result:", photo_post_result)


# Utils Tests

##  def get_page_or_user_info
##  def get_page_id (scraping)



In [ ]:
user_info = Utils.get_page_or_user_info(api_client=fb_client, page_or_user_id="100044233849394")
print(user_info)

# Fetch specific fields
specific_fields = ["id", "name", "link", "website"]
page_info = Utils.get_page_or_user_info(api_client=fb_client, page_or_user_id=user_id, fields=specific_fields)
print(page_info)

## Get the id of a page by its name

In [ ]:
page_name = "laesenciadehuelva"  # Replace with the actual Page name
user_id = Utils.get_page_id(page_name)

if user_id:
    print(f"Found User ID for {page_name}: {user_id}")
else:
    print(f"Could not find User ID for {page_name}.")